In [2]:
!pip install neo4j

In [3]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.1.1


In [4]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [5]:
conn = Neo4jConnection(uri="neo4j://localhost:7687", user="neo4j", pwd="graph")

In [8]:
conn.query("MATCH (n1)-[r]->(n2) RETURN r, n1, n2 LIMIT 5")

[<Record r=<Relationship id=29549 nodes=(<Node id=0 labels=frozenset({'TEAM_ID'}) properties={'id': 1610612762}>, <Node id=3559 labels=frozenset({'LEAGUE_ID'}) properties={'id': 0}>) type='IN_LEAGUE' properties={}> n1=<Node id=0 labels=frozenset({'TEAM_ID'}) properties={'id': 1610612762}> n2=<Node id=3559 labels=frozenset({'LEAGUE_ID'}) properties={'id': 0}>>,
 <Record r=<Relationship id=29550 nodes=(<Node id=0 labels=frozenset({'TEAM_ID'}) properties={'id': 1610612762}>, <Node id=3788 labels=frozenset({'CITY'}) properties={'name': 'UTAH'}>) type='IN' properties={}> n1=<Node id=0 labels=frozenset({'TEAM_ID'}) properties={'id': 1610612762}> n2=<Node id=3788 labels=frozenset({'CITY'}) properties={'name': 'UTAH'}>>,
 <Record r=<Relationship id=29551 nodes=(<Node id=0 labels=frozenset({'TEAM_ID'}) properties={'id': 1610612762}>, <Node id=3789 labels=frozenset({'ARENA'}) properties={'name': 'VIVINT SMART HOME ARENA'}>) type='TO' properties={}> n1=<Node id=0 labels=frozenset({'TEAM_ID'}) pro

In [7]:
conn.query("""
    MATCH (p:PLAYER_ID)-[:POINTS]->(n:PTS)<-[:IN_GAME_PTS]-(g:GAME_ID)
    MATCH (p)-[:HAS_NAME]->(m:PLAYER_NAME)
    RETURN m AS player,
      AVG(n.points) AS points
    ORDER BY points DESC
    LIMIT 10
;
    """)

[<Record player=<Node id=1000 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'KEVIN DURANT'}> points=26.37535277516474>,
 <Record player=<Node id=417 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'LEBRON JAMES'}> points=26.11947728686998>,
 <Record player=<Node id=2233 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'KOBE BRYANT'}> points=24.761343012704163>,
 <Record player=<Node id=691 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'JAMES HARDEN'}> points=23.909359605911263>,
 <Record player=<Node id=398 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'LUKA DONCIC'}> points=23.26923076923078>,
 <Record player=<Node id=753 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'DAMIAN LILLARD'}> points=23.215189873417792>,
 <Record player=<Node id=956 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'STEPHEN CURRY'}> points=22.733556298773713>,
 <Record player=<Node id=3462 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'ALLEN IVERSON'}> po

In [12]:
conn.query("""
MATCH (p:PLAYER_ID)-[:HAS_NAME]-(m:PLAYER_NAME)
WHERE m.name = 'CHRIS PAUL'
MATCH (p)-[:FG_ATTEMPTED]->(fga:FGA)
MATCH (p)-[:FG_PCT]->(fg:FG_PCT)
RETURN m,
       AVG(fg.fg_pct) AS shots,
       AVG(fga.fg_attempted) AS attempted,
       SUM(fga.fg_attempted) AS total
;
""")

[<Record m=<Node id=504 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'CHRIS PAUL'}> shots=0.4255745341615994 attempted=12.577639751554061 total=229521600>]

In [6]:
conn.query("MATCH (m:PLAYER_NAME {name:'LEBRON JAMES'}) MATCH (p:PLAYER_ID)-[:HAS_NAME]->(m) MATCH (t:TEAM_ID)-[:HAS_NICKNAME]->(n:NICKNAME) MATCH (p)-[:PLAYS_AT]->(t) RETURN n")

[<Record n=<Node id=3650 labels=frozenset({'NICKNAME'}) properties={'name': 'LAKERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3650 labels=frozenset({'NICKNAME'}) properties={'name': 'LAKERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<No

In [10]:
conn.query("""
MATCH (p)-[:HAS_NAME]->(m:PLAYER_NAME)
RETURN m AS player, COUNT(p) AS count
ORDER BY count DESC
LIMIT 10
;
""")

[<Record player=<Node id=1862 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'CHRIS JOHNSON'}> count=18>,
 <Record player=<Node id=612 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'ISH SMITH'}> count=17>,
 <Record player=<Node id=1361 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'COREY BREWER'}> count=16>,
 <Record player=<Node id=346 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'GARRETT TEMPLE'}> count=16>,
 <Record player=<Node id=92 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'ERSAN ILYASOVA'}> count=15>,
 <Record player=<Node id=86 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'WESLEY MATTHEWS'}> count=15>,
 <Record player=<Node id=769 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'ANTHONY TOLLIVER'}> count=14>,
 <Record player=<Node id=1367 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'JERRYD BAYLESS'}> count=14>,
 <Record player=<Node id=560 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'D.J. AUGUSTIN'}>

In [11]:
conn.query("""
MATCH (g:GAME_ID)-[:TEAM_HOME]->(t:TEAM_ID)-[:HAS_NICKNAME]->(n:NICKNAME {name:'NUGGETS'})
MATCH (g)-[:HAS_FG3_H]->(fg:FG3_PCT_home)
RETURN n AS team,
      AVG(fg.fg3_pct_H) AS fg3
;
""")

[<Record team=<Node id=3618 labels=frozenset({'NICKNAME'}) properties={'name': 'NUGGETS'}> fg3=0.3483071808510635>]

In [8]:
conn.query("""
MATCH (m:PLAYER_NAME {name:'LEBRON JAMES'})
MATCH (p:PLAYER_ID)-[:HAS_NAME]->(m)
MATCH (t:TEAM_ID)-[:HAS_NICKNAME]->(n:NICKNAME)
MATCH (p)-[:PLAYS_AT]->(t)
RETURN n
;
""")

[<Record n=<Node id=3650 labels=frozenset({'NICKNAME'}) properties={'name': 'LAKERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3650 labels=frozenset({'NICKNAME'}) properties={'name': 'LAKERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3658 labels=frozenset({'NICKNAME'}) properties={'name': 'HEAT'}>>,
 <Record n=<Node id=3834 labels=frozenset({'NICKNAME'}) properties={'name': 'CAVALIERS'}>>,
 <Record n=<No

In [9]:
conn.query("""
MATCH (p:PLAYER_ID)-[:START_POS]->(s:START_POSITION)
WHERE s.start_pos <> '-'
MATCH (p)-[:HAS_NAME]->(m:PLAYER_NAME)
RETURN m AS player,
       COUNT(p) AS count
ORDER BY count DESC
LIMIT 5
;
""")

[<Record player=<Node id=417 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'LEBRON JAMES'}> count=16610>,
 <Record player=<Node id=747 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'CARMELO ANTHONY'}> count=14328>,
 <Record player=<Node id=930 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'PAU GASOL'}> count=13620>,
 <Record player=<Node id=1128 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'JOE JOHNSON'}> count=13260>,
 <Record player=<Node id=435 labels=frozenset({'PLAYER_NAME'}) properties={'name': 'DWIGHT HOWARD'}> count=12716>]